<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">

# Derivatives Analytics with Python
**&mdash;Part II: Arbitrage Pricing**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>

## Part II &mdash; Arbitrage Pricing

### Chapter 6 &mdash; Fourier-Based Option Pricing

Fourier methods are a practical bridge between model specification and valuation when the
risk-neutral distribution is best described by a characteristic function rather than an
explicit density.

The chapter develops two key approaches:

- Lewis-style inversion integrals for option values.
- Carr&mdash;Madan pricing on a strike grid via the FFT.

Reference notebooks and scripts for the book are available under `x_store/dawp/python36`.

The notebook is designed to run smoothly on Google Colab. A local setup is optional and
can be based on `python -m venv .venv`.

### 1. Environment check

We start by importing the core libraries for this class and by printing version
information to confirm that a modern Python environment is active.

In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for optional figure export

import math  # elementary math functions

import numpy as np  # numerical arrays and linear algebra tools
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

from scipy import stats  # normal distribution functions
from scipy.integrate import quad  # numerical integration

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import (  # Jupyter helper
        set_matplotlib_formats,
    )
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline rendering


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")

### 2. Pricing with characteristic functions

In many models, the distribution of the log-return $X_T = \log(S_T / S_0)$ is naturally
described by its characteristic function
$$\varphi_T(u) = \mathbb{E}^{\mathbb{Q}}[e^{iuX_T}].$$

Fourier methods build option values from $\varphi_T$ without explicitly constructing the
density of $X_T$.

### 3. BSM benchmark: analytical call value and characteristic function

We use the Black&mdash;Scholes&mdash;Merton model as a benchmark. It provides both a closed-form
call value and a closed-form characteristic function for the log-return.

In [ ]:
def bsm_call_value(S0, K, T, r, sigma):
    # Analytical BSM European call value (no dividends).
    if T <= 0.0:
        return max(S0 - K, 0.0)
    if sigma <= 0.0:
        return math.exp(-r * T) * max(S0 - K, 0.0)

    vol_sqrt_T = sigma * math.sqrt(T)  # volatility scaling term
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma * sigma) * T) / vol_sqrt_T
    d2 = d1 - vol_sqrt_T  # d2 term

    pvK = K * math.exp(-r * T)  # discounted strike
    return S0 * stats.norm.cdf(d1) - pvK * stats.norm.cdf(d2)


def bsm_char_func(u, T, r, sigma):
    # Characteristic function of X_T = log(S_T / S_0) under risk-neutral dynamics.
    drift = (r - 0.5 * sigma * sigma) * T  # drift of log-return
    var = sigma * sigma * T  # variance of log-return
    return np.exp(1j * u * drift - 0.5 * var * u * u)


def bsm_char_func_ln_st(u, S0, T, r, sigma):
    # Characteristic function of ln(S_T) under risk-neutral dynamics.
    x0 = math.log(S0)  # initial log-level
    return np.exp(1j * u * x0) * bsm_char_func(u, T, r, sigma)


S0 = 100.0  # initial underlying level
K = 100.0  # strike level
T = 1.0  # time to maturity in years
r = 0.05  # constant short rate
sigma = 0.20  # volatility

print(f"BSM call value: {bsm_call_value(S0, K, T, r, sigma):.6f}")

### 4. Lewis integral pricing

Lewis-style pricing expresses the call value as a single integral involving the
characteristic function evaluated at a complex argument. The shift ensures integrability
for diffusion models like BSM.

In [ ]:
def lewis_integrand(u, S0, K, T, r, sigma):
    # Integrand for the Lewis (2001) call pricing formula.
    k = math.log(S0 / K)  # log-moneyness in the exponential term
    cf = bsm_char_func(u - 0.5j, T, r, sigma)  # shifted characteristic function
    num = np.exp(1j * u * k) * cf  # complex numerator
    den = u * u + 0.25  # integrability denominator
    return (num / den).real


def bsm_call_value_lewis(S0, K, T, r, sigma, u_max=100.0):
    # BSM call value via the Lewis (2001) Fourier integral representation.
    integral = quad(
        lambda u: lewis_integrand(u, S0, K, T, r, sigma),
        0.0,
        u_max,
    )[0]  # numerical integral approximation

    factor = math.exp(-r * T) * math.sqrt(S0 * K) / math.pi  # prefactor
    value = S0 - factor * integral  # Lewis call value
    return max(0.0, float(value))


print(f"Lewis call value: {bsm_call_value_lewis(S0, K, T, r, sigma):.6f}")

### 5. Carr&mdash;Madan FFT pricing on a strike grid

Carr&mdash;Madan pricing computes a grid of call values by applying the FFT to a damped
Fourier transform. This is efficient when many strikes must be valued at once.

In [ ]:
from numpy.fft import fft  # fast Fourier transform


def carr_madan_call_grid(S0, T, r, sigma, alpha=1.5, N=4096, eta=0.25):
    # Returns (K_grid, C_grid) via Carr-Madan FFT pricing.
    lam = 2.0 * math.pi / (N * eta)  # log-strike spacing
    b = 0.5 * N * lam  # upper bound for log-strikes

    v = eta * np.arange(N)  # Fourier grid
    k = -b + lam * np.arange(N)  # log-strike grid (log(K/S0))

    u = v - 1j * (alpha + 1.0)  # complex CF argument
    cf = bsm_char_func_ln_st(u, S0, T, r, sigma)  # characteristic function values

    denom = (alpha * alpha + alpha - v * v) + 1j * (2.0 * alpha + 1.0) * v
    psi = math.exp(-r * T) * cf / denom  # transform of damped call

    weights = np.ones(N)  # trapezoidal weights
    weights[0] = 0.5  # half weight at the first point

    integrand = np.exp(1j * b * v) * psi * eta * weights
    fft_vals = fft(integrand).real  # real FFT values
    C = np.exp(-alpha * k) / math.pi * fft_vals  # undamp

    K_grid = np.exp(k)  # strike grid
    return K_grid, C


K_fft, C_fft = carr_madan_call_grid(S0, T, r, sigma)  # FFT strike/value grid
print(K_fft[:5])
print(C_fft[:5])

### 6. Comparison figure for the slide deck

We compare analytical, Lewis integral, and FFT values on a strike grid and export the
result as a PDF figure for the Part II slide deck.

In [ ]:
K_grid = np.linspace(60.0, 140.0, 41)  # strike grid for comparison

ana = np.array([bsm_call_value(S0, k, T, r, sigma) for k in K_grid])
lewis = np.array([bsm_call_value_lewis(S0, k, T, r, sigma) for k in K_grid])
fft_vals = np.interp(K_grid, K_fft, C_fft)  # interpolate FFT grid to K_grid

rel_err_lewis = (lewis - ana) / np.maximum(ana, 1e-12)  # relative error
rel_err_fft = (fft_vals - ana) / np.maximum(ana, 1e-12)  # relative error

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)  # two-row figure

ax1.plot(K_grid, ana, lw=2.0, label="analytical")
ax1.plot(K_grid, lewis, lw=2.0, ls="--", label="Lewis integral")
ax1.plot(K_grid, fft_vals, lw=2.0, ls=":", label="Carr-Madan FFT")
ax1.set_ylabel("call value")
ax1.set_title("BSM call values: analytical vs Fourier methods")
ax1.legend(loc=0)

ax2.plot(K_grid, rel_err_lewis, lw=1.8, label="Lewis rel. error")
ax2.plot(K_grid, rel_err_fft, lw=1.8, label="Carr-Madan rel. error")
ax2.axhline(0.0, lw=1.0, color="black")
ax2.set_xlabel("strike K")
ax2.set_ylabel("relative error")
ax2.legend(loc=0)

fig.tight_layout()
maybe_save(fig, "dawp_pII_fig04_fou_compare")  # optional PDF export
plt.show()

### 7. Roots of unity

The FFT evaluates a discrete Fourier transform at the roots of unity. We visualize them on
the unit circle and export the figure for the slide deck.

In [ ]:
N = 16  # number of roots of unity
m = np.arange(N)  # root indices
z = np.exp(2.0j * np.pi * m / N)  # roots of unity on the unit circle

fig, ax = plt.subplots()  # create a single plot
ax.plot(np.real(z), np.imag(z), "o", ms=7)  # plot the roots

theta = np.linspace(0.0, 2.0 * np.pi, 400)  # unit circle parameter grid
ax.plot(np.cos(theta), np.sin(theta), lw=1.2)  # draw the unit circle

ax.set_aspect("equal", "box")  # equal scaling in x and y
ax.set_xlabel("real part")  # x-axis label
ax.set_ylabel("imaginary part")  # y-axis label
ax.set_title(f"Roots of unity (N={N})")  # plot title

maybe_save(fig, "dawp_pII_fig05_roots_unity")  # optional PDF export
plt.show()

### 8. Summary

Fourier methods price options directly from characteristic functions. Lewis-style integrals
provide a robust single-integral representation, and Carr&mdash;Madan pricing leverages the
FFT to compute a whole strike grid efficiently.

<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">